In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
test = pd.read_csv('../input/test.csv')

In [ ]:
qid_test = pd.read_csv('../input/test.csv')[['qid']]

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Classification with fastai

In [ ]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
df = pd.DataFrame({'label':train.target, 'text':train.question_text})

In [ ]:
test = pd.DataFrame({'text':test.question_text})

In [ ]:
# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.25, random_state = 42)

In [ ]:
test.head()

In [ ]:
df_trn.head()

In [ ]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, test_df=test, path = "", text_cols='text', label_cols='label')

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, test_df = test,text_cols='text',label_cols='label', vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM)

In [ ]:
moms = (0.8,0.7)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-2), moms=moms)

In [ ]:
learn.save_encoder('enc')

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('enc')
learn.fit_one_cycle(1, moms=moms)

In [ ]:
res = learn.get_preds(ds_type=DatasetType.Valid)

In [ ]:
# check f1 score on valid
predictions = np.argmax(res[0], axis=1)
f1_score(res[1], predictions) # 0.537

In [ ]:
pd.crosstab(predictions, res[1])

In [ ]:
# check on test set
test_res = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
test_pred = np.argmax(test_res[0], axis=1)

In [ ]:
submission_df = pd.DataFrame({'qid':qid_test.qid, 'prediction':test_pred})

In [ ]:
submission_df.to_csv('submission.csv', index=False)